In [ ]:
!pip install nlp-utils

In [ ]:
import nltk
import string
import pandas as pd
import nlp_utils as nu
import matplotlib.pyplot as plt
# Loading necessary libraries

In [ ]:
f = open("/content/dialogs.txt", "r")
print(f.read())
# reading the data

In [ ]:
df=pd.read_csv('/content/dialogs.txt',names=('Query','Response'),sep=('\t'))
# Reading the data

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
Text=df['Query']

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
sid = SentimentIntensityAnalyzer()
for sentence in Text:
     print(sentence)

     ss = sid.polarity_scores(sentence)
     for k in ss:
         print('{0}: {1}, ' .format(k, ss[k]), end='')
     print()

In [ ]:
analyzer = SentimentIntensityAnalyzer()
df['rating'] = Text.apply(analyzer.polarity_scores)
df=pd.concat([df.drop(['rating'], axis=1), df['rating'].apply(pd.Series)], axis=1)
### Creating a dataframe.

In [ ]:
from wordcloud import WordCloud
# importing word cloud

In [ ]:
def wordcloud(df, label):

    subset=df[df[label]==1]
    text=df.Query.values
    wc= WordCloud(background_color="black",max_words=1000)

    wc.generate(" ".join(text))

    plt.figure(figsize=(20,20))
    plt.subplot(221)
    plt.axis("off")
    plt.title("Words frequented in {}".format(label), fontsize=20)
    plt.imshow(wc.recolor(colormap= 'gist_earth' , random_state=244), alpha=0.98)
# visualising wordcloud

In [ ]:
import re

In [ ]:
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [ ]:
remove_n = lambda x: re.sub("\n", " ", x)

In [ ]:
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)

In [ ]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)

In [ ]:
df['Query'] = df['Query'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

In [ ]:
df['Response'] = df['Response'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

In [ ]:
pd.set_option('display.max_rows',3800)

In [ ]:
imp_sent=df.sort_values(by='compound', ascending=False)

In [ ]:
pos_sent=df.sort_values(by='pos', ascending=False)

In [ ]:
neg_sent=df.sort_values(by='neg', ascending=False)

In [ ]:
neu_sent=df.sort_values(by='neu', ascending=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
factors = tfidf.fit_transform(df['Query']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
query = 'who are you ?'
def chatbot(query):
    # step:-1 clean
    query = nu.lemmatization_sentence(query)
    # step:-2 word embedding - transform
    query_vector = tfidf.transform([query]).toarray()
    # step-3: cosine similarity
    similar_score = 1 -cosine_distances(factors,query_vector)
    index = similar_score.argmax() # take max index position
    # searching or matching question
    matching_question = df.loc[index]['Query']
    response = df.loc[index]['Response']
    pos_score = df.loc[index]['pos']
    neg_score = df.loc[index]['neg']
    neu_score = df.loc[index]['neu']
    confidence = similar_score[index][0]
    chat_dict = {'match':matching_question,
                'response':response,
                'score':confidence,
                'pos':pos_score,
                'neg':neg_score,
                'neu':neu_score}
    return chat_dict

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
while True:
    query = input('USER: ')
    if query == 'exit':
        break

    response = chatbot(query)
    if response['score'] <= 0.2: #
        print('BOT: Please rephrase your Question.')

    else:
        print('='*80)
        print('logs:\n Matched Question: %r\n Confidence Score: %0.2f \n PositiveScore: %r \n NegativeScore: %r\n NeutralScore: %r'%(
            response['match'],response['score']*100,response['pos'],response['neg'],response['neu']))
        print('='*80)
        print('BOT: ',response['response'])